In [ ]:
import pandas as pd

In [ ]:
sequences = pd.read_csv("../data/ml-1m/raw_sequences.csv")

In [ ]:
sequences

In [ ]:
# テストデータに含まれる顧客の学習データが何個かあるかによって学習データを分けたい
# 学習データがx個ある顧客のみを含んだテストデータ
# x = [10, 20, 30, 40, 50]
sequences.movie_id = sequences.movie_id.apply(lambda s: s.split(" "))

In [ ]:
# 各系列ごとの末尾n個をテストデータとする
import random
from typing import Dict, List, Tuple
random.seed(0)

a = []

counter = {}
counter[-1] = 0
for i in range(5 + 1):
    counter[i * 10] = 0

for _, (user_id, movie_id) in sequences.iterrows():
    seq = movie_id
    seq_l = len(seq)

    while True:
        l = random.randint(-1, 5)
        if l == -1:
            # そのまま採用する
            train_seq = seq
            test_seq = []
            train_l = -1
            counter[train_l] += 1
            a.append((user_id, train_l, train_seq, test_seq))
            break
        else:
            train_l = l * 10
            if counter[train_l] == 500:
                continue

            test_l = 20
            required_l = train_l + test_l
            if seq_l < required_l:
                continue
            train_seq = seq[-required_l:-test_l]
            test_seq = seq[-test_l:]
            counter[train_l] += 1
            a.append((user_id, train_l, train_seq, test_seq))
            break

In [ ]:
train_sequences = {}
test_sequences = {}
all_test_sequences = {}

test_size = [0, 10, 20, 30, 40, 50]
for sz in test_size:
    test_sequences[sz] = {}

for user_id, train_l, train_seq, test_seq in a:
    assert user_id not in train_sequences
    train_sequences[user_id] = " ".join(train_seq)
    if train_l >= 0:
        all_test_sequences[user_id] = " ".join(test_seq)
        test_sequences[train_l][user_id] = " ".join(test_seq)

for sz in test_size:
    file_path = f"../data/ml-1m-new/test-{sz}.csv"
    test_sz_df = pd.DataFrame.from_dict(test_sequences[sz], orient="index", columns=["sequence"])
    test_sz_df.index.name = "user_id"
    test_sz_df.to_csv(file_path)

In [ ]:
train_df = pd.DataFrame.from_dict(train_sequences, orient="index", columns=["sequence"])
train_df.index.name = "user_id"
train_df.to_csv("../data/ml-1m-new/train.csv")

In [ ]:
test_df = pd.DataFrame.from_dict(all_test_sequences, orient="index", columns=["sequence"])
test_df.index.name = "user_id"
test_df.to_csv("../data/ml-1m-new/test.csv")